In [1]:
import pandas as pd
import os
import math
import time

## Step 1: Separate Left and Right Hand Video Segments

In [2]:
def create_directory(file_path):
    if os.path.exists(file_path):
        return
    
    parent = os.path.dirname(file_path)
    
    if os.path.exists(parent):
        os.mkdir(file_path)
        return
    
    create_directory(parent)
    os.mkdir(file_path)
    return

def number_to_time(time_in_fraction):
    '''
    input: 0.45 = 45 seconds
    output: 00:00:45 (hh:mm:ss)
    '''
    seconds = round(time_in_fraction*100,2)
    assert(seconds<60)
    if seconds<10:
        time = "00:00:0"+str(seconds)
    else:
        time = "00:00:"+str(seconds)
    return time

In [3]:
#Read Excel Sheet
INPUT_LR_SPLIT_FILE = "E:/Saiful/park_motor/LeftRight/data/right_left_label.csv"
#Original Task2 Videos
INPUT_TASK2_VIDEOS = "E:/Saiful/park_motor/Task2_All"
#This is where separated videos will go
OUTPUT_LEFT_VIDEOS = "E:/Saiful/park_motor/LeftRight/Task2_Videos/Left"
OUTPUT_RIGHT_VIDEOS = "E:/Saiful/park_motor/LeftRight/Task2_Videos/Right"

#Create directories if not present
create_directory(OUTPUT_LEFT_VIDEOS)
create_directory(OUTPUT_RIGHT_VIDEOS)

error_files = []

temp_dir = os.path.join(OUTPUT_LEFT_VIDEOS, "temp")
os.mkdir(temp_dir)

In [4]:
success = 0
failed = 0

df = pd.read_csv(INPUT_LR_SPLIT_FILE)
for index, row in df.iterrows():
    
    filename = row["File_name"]
    left_start = row["left_begin"]
    left_end = row["left_end"]
    right_start = row[" right_begin"]
    right_end = row["right_end"]
    
    input_file_path = os.path.join(INPUT_TASK2_VIDEOS, filename)
    #Make sure it exists
    assert(os.path.exists(input_file_path))
    
    output_left_filename = filename[:-5]+"-left"+".mp4"
    output_right_filename = filename[:-5]+"-right"+".mp4"
    
    output_left_path = os.path.join(OUTPUT_LEFT_VIDEOS, output_left_filename)
    output_right_path = os.path.join(OUTPUT_RIGHT_VIDEOS, output_right_filename)
    
    
    temp_output_file = filename[:-5]+"-temp"+".mp4"
    temp_path = os.path.join(temp_dir, temp_output_file)
    
    #Resize to 256x256, convert to mp4
    command = "ffmpeg -i \"%s\" -vf scale=256:256 \"%s\""%(input_file_path, temp_path)
    out1 = os.system(command)
    print(out1)
    
    if out1!=0:
        print(command)
        error_files.append(output_left_path)
        error_files.append(output_right_path)
        failed +=2
    else:
        time.sleep(2)
    #Sample command to cut video by time
    #ffmpeg -i "E:/Saiful/park_motor/Task2_All/2017-08-18T14-59-52-530Z49-task2.webm" -ss 00:00:03 -t 00:00:08 -async 1 -c copy "E:/Saiful/park_motor/LeftRight/Task2_Videos/Test/test.webm"
    
    if out1==0 and (left_start!=(-1.0)) and (left_end!=(-1.0)):
        #Clip left part
        if not os.path.exists(output_left_path):
            left_start_time = number_to_time(left_start)
            left_end_time = number_to_time(left_end)
            command = "ffmpeg -i \"%s\" -ss %s -t %s -async 1 -c copy \"%s\""%(temp_path, left_start_time, left_end_time, output_left_path)
            print(command)
            out = os.system(command)
            print(out)
            if out!=0:
                print(command)
                error_files.append(output_left_path)
                failed +=1
            else:
                success +=1
                time.sleep(2)
            
        
    if out1==0 and (right_start!=(-1.0)) and (right_end!=(-1.0)):
        #Clip right part
        if not os.path.exists(output_right_path):
            right_start_time = number_to_time(right_start)
            right_end_time = number_to_time(right_end)
            command = "ffmpeg -i \"%s\" -ss %s -t %s -async 1 -c copy \"%s\""%(temp_path, right_start_time, right_end_time, output_right_path)
            print(command)
            out = os.system(command)
            print(out)
            if out!=0:
                print(command)
                error_files.append(output_left_path)
                failed +=1
            else:
                success +=1
                time.sleep(2)
                
    print("Attempted %d, success %d, failed %d \n"%((failed+success), success, failed))

0
ffmpeg -i "E:/Saiful/park_motor/LeftRight/Task2_Videos/Left\temp\2017-08-18T14-59-52-530Z49-task2-temp.mp4" -ss 00:00:02.0 -t 00:00:08.0 -async 1 -c copy "E:/Saiful/park_motor/LeftRight/Task2_Videos/Left\2017-08-18T14-59-52-530Z49-task2-left.mp4"
0
ffmpeg -i "E:/Saiful/park_motor/LeftRight/Task2_Videos/Left\temp\2017-08-18T14-59-52-530Z49-task2-temp.mp4" -ss 00:00:08.0 -t 00:00:16.0 -async 1 -c copy "E:/Saiful/park_motor/LeftRight/Task2_Videos/Right\2017-08-18T14-59-52-530Z49-task2-right.mp4"
0
Attempted 2, success 2, failed 0 

0
ffmpeg -i "E:/Saiful/park_motor/LeftRight/Task2_Videos/Left\temp\2017-08-18T15-24-14-004Z53-task2-temp.mp4" -ss 00:00:11.0 -t 00:00:28.0 -async 1 -c copy "E:/Saiful/park_motor/LeftRight/Task2_Videos/Left\2017-08-18T15-24-14-004Z53-task2-left.mp4"
0
ffmpeg -i "E:/Saiful/park_motor/LeftRight/Task2_Videos/Left\temp\2017-08-18T15-24-14-004Z53-task2-temp.mp4" -ss 00:00:00.0 -t 00:00:11.0 -async 1 -c copy "E:/Saiful/park_motor/LeftRight/Task2_Videos/Right\2017-08

Attempted 31, success 31, failed 0 

0
ffmpeg -i "E:/Saiful/park_motor/LeftRight/Task2_Videos/Left\temp\2017-10-17T02-00-21-466Z4-task2-temp.mp4" -ss 00:00:00.0 -t 00:00:07.0 -async 1 -c copy "E:/Saiful/park_motor/LeftRight/Task2_Videos/Left\2017-10-17T02-00-21-466Z4-task2-left.mp4"
0
ffmpeg -i "E:/Saiful/park_motor/LeftRight/Task2_Videos/Left\temp\2017-10-17T02-00-21-466Z4-task2-temp.mp4" -ss 00:00:07.0 -t 00:00:17.0 -async 1 -c copy "E:/Saiful/park_motor/LeftRight/Task2_Videos/Right\2017-10-17T02-00-21-466Z4-task2-right.mp4"
0
Attempted 33, success 33, failed 0 

0
ffmpeg -i "E:/Saiful/park_motor/LeftRight/Task2_Videos/Left\temp\2017-10-20T17-52-20-385Z99-task2-temp.mp4" -ss 00:00:00.0 -t 00:00:04.0 -async 1 -c copy "E:/Saiful/park_motor/LeftRight/Task2_Videos/Left\2017-10-20T17-52-20-385Z99-task2-left.mp4"
0
ffmpeg -i "E:/Saiful/park_motor/LeftRight/Task2_Videos/Left\temp\2017-10-20T17-52-20-385Z99-task2-temp.mp4" -ss 00:00:04.0 -t 00:00:11.0 -async 1 -c copy "E:/Saiful/park_motor/L

Attempted 62, success 62, failed 0 

0
ffmpeg -i "E:/Saiful/park_motor/LeftRight/Task2_Videos/Left\temp\2017-11-21T00-12-19-685Z69-task2-temp.mp4" -ss 00:00:03.0 -t 00:00:07.0 -async 1 -c copy "E:/Saiful/park_motor/LeftRight/Task2_Videos/Left\2017-11-21T00-12-19-685Z69-task2-left.mp4"
0
ffmpeg -i "E:/Saiful/park_motor/LeftRight/Task2_Videos/Left\temp\2017-11-21T00-12-19-685Z69-task2-temp.mp4" -ss 00:00:00.0 -t 00:00:03.0 -async 1 -c copy "E:/Saiful/park_motor/LeftRight/Task2_Videos/Right\2017-11-21T00-12-19-685Z69-task2-right.mp4"
0
Attempted 64, success 64, failed 0 

0
ffmpeg -i "E:/Saiful/park_motor/LeftRight/Task2_Videos/Left\temp\2017-11-21T02-20-37-013Z74-task2-temp.mp4" -ss 00:00:03.0 -t 00:00:08.0 -async 1 -c copy "E:/Saiful/park_motor/LeftRight/Task2_Videos/Left\2017-11-21T02-20-37-013Z74-task2-left.mp4"
0
ffmpeg -i "E:/Saiful/park_motor/LeftRight/Task2_Videos/Left\temp\2017-11-21T02-20-37-013Z74-task2-temp.mp4" -ss 00:00:00.0 -t 00:00:03.0 -async 1 -c copy "E:/Saiful/park_mot

ffmpeg -i "E:/Saiful/park_motor/LeftRight/Task2_Videos/Left\temp\2017-12-06T03-25-20-889Z90-task2-temp.mp4" -ss 00:00:00.0 -t 00:00:03.0 -async 1 -c copy "E:/Saiful/park_motor/LeftRight/Task2_Videos/Right\2017-12-06T03-25-20-889Z90-task2-right.mp4"
0
Attempted 94, success 94, failed 0 

0
ffmpeg -i "E:/Saiful/park_motor/LeftRight/Task2_Videos/Left\temp\2017-12-06T07-04-56-859Z15-task2-temp.mp4" -ss 00:00:04.0 -t 00:00:09.0 -async 1 -c copy "E:/Saiful/park_motor/LeftRight/Task2_Videos/Left\2017-12-06T07-04-56-859Z15-task2-left.mp4"
0
ffmpeg -i "E:/Saiful/park_motor/LeftRight/Task2_Videos/Left\temp\2017-12-06T07-04-56-859Z15-task2-temp.mp4" -ss 00:00:00.0 -t 00:00:04.0 -async 1 -c copy "E:/Saiful/park_motor/LeftRight/Task2_Videos/Right\2017-12-06T07-04-56-859Z15-task2-right.mp4"
0
Attempted 96, success 96, failed 0 

0
ffmpeg -i "E:/Saiful/park_motor/LeftRight/Task2_Videos/Left\temp\2017-12-06T12-34-54-878Z70-task2-temp.mp4" -ss 00:00:00.0 -t 00:00:04.0 -async 1 -c copy "E:/Saiful/park_m

Attempted 124, success 124, failed 0 

0
ffmpeg -i "E:/Saiful/park_motor/LeftRight/Task2_Videos/Left\temp\2017-12-20T14-45-51-894Z87-task2-temp.mp4" -ss 00:00:04.0 -t 00:00:12.0 -async 1 -c copy "E:/Saiful/park_motor/LeftRight/Task2_Videos/Left\2017-12-20T14-45-51-894Z87-task2-left.mp4"
0
ffmpeg -i "E:/Saiful/park_motor/LeftRight/Task2_Videos/Left\temp\2017-12-20T14-45-51-894Z87-task2-temp.mp4" -ss 00:00:00.0 -t 00:00:04.0 -async 1 -c copy "E:/Saiful/park_motor/LeftRight/Task2_Videos/Right\2017-12-20T14-45-51-894Z87-task2-right.mp4"
0
Attempted 126, success 126, failed 0 



KeyboardInterrupt: 